# Load data

In [58]:
import pandas as pd
import requests
from requests.adapters import HTTPAdapter, Retry

# Formatting the coordinates into a format recognised by the API
mrt_df = pd.read_csv("mrt_lrt_data.csv")
mrt_df["coord"] = mrt_df["lat"].astype(str) +"%2C"+ mrt_df["lng"].astype(str)

In [277]:
street_df =  pd.read_csv("ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv")
street_df = street_df["street_name"].unique()
street_df = pd.DataFrame(street_df)

# Google Maps API

Documentation: https://developers.google.com/maps/documentation/geocoding/overview


This section will geocode the street names into coordinates

In [274]:
google_api_key = "xxxxx"

**Setting up the API**

In [275]:
google_geocode_api = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 502, 503, 504, 400, "ZERO_RESULTS" ])
google_geocode_api.mount('http://', HTTPAdapter(max_retries=retries))

**Transforming some addresses:**

The Google Maps API was not able to find some streets because it contained a few shortform spellings of certain words, for example "kampong" or "jalan" was written as "KG" and "JLN" respectively. On top of that some streets names just could not be retrieved, so a nearby street was used to replace it as an approximation

In [349]:
def get_coord(address):
    address = address.replace("EDGEFIELD PLAINS", "Punggol Central")
    address = address.replace("HENDERSON RD", "Bukit Merah Central")
    address = address.replace("DEPOT RD", "Bukit Merah Central")
    address = address.replace("JLN ", "Jalan ")
    address = address.replace("KG ", "Kampong ")
    address = address.replace(" NTH ", " North ")
    address = address.replace("NTH ", "North ")
    address = address.replace(" DR", " Drive")
    address = address.replace("QUEEN ST", "MIDDLE ROAD")
    address = address.replace(" RD", " Road")
    address = address.replace("BEACH ROAD", "BEACH RD")
    address = address.replace("CAMBRIDGE ROAD", "CAMBRIDGE ROAD SINGAPORE")
    address = address.replace(" PL", " Place")
    address = address.replace(" CL", " Close")
    address = address.replace(" ST", " Street")
    address = address.replace("MARINE TER", "MARINE CRES")
    address = address.replace(" TER", " Terrace")
    address = address.replace("C'WEALTH", "Commonwealth")
    url_coord =  "https://maps.googleapis.com/maps/api/geocode/json?address="+address+" singapore&region=SG"+"&key=" + google_api_key+"&components=route"
    response_coord = google_geocode_api.get(url_coord)
    response_coord = response_coord.json()
    return response_coord["results"]

In [351]:
street_df["coordinates"] = street_df[0].apply(get_coord)
street_df.head()

In [353]:
import json
street_df["coordinates"] = street_df["coordinates"].apply(lambda x: x[0] if len(x) > 0 else None)
geometry = street_df["coordinates"].apply(lambda x: x.get("geometry") if isinstance(x, dict) else None)
lat_long = geometry.apply(lambda x: x.get("location") if isinstance(x, dict) else None)
street_df["latitude"]= lat_long.apply(lambda x: x.get("lat") if isinstance(x, dict) else None)
street_df["longitude"]= lat_long.apply(lambda x: x.get("lng") if isinstance(x, dict) else None)
street_df["lat_long"] = street_df["latitude"].astype(str) +"%2C"+ street_df["longitude"].astype(str)


**Save street coordinates to CSV**

In [354]:
street_df.to_csv("street_coord.csv")

# Find the closest MRT station for each street

Normally, the Haversine formula is used to calculate the distance between coordinates, which takes into consideration the curvature of the Earth. However, for ease of coding, and also because Singapore is so small, taking a simple Euclidean distance suffices.

In [356]:
street_df_complete = street_df
mrt_df['coord_tuple'] = mrt_df.apply(lambda row: [row['lat'], row['lng']], axis=1)
street_df_complete['coord_tuple'] = street_df_complete.apply(lambda row: [row['latitude'], row['longitude']], axis=1)

In [357]:
from scipy.spatial import distance

# function to calculate closest euclidean distance
def closest_mrt(coord_tuple):
    return min(mrt_df["coord_tuple"], key=lambda x: distance.euclidean(x, coord_tuple))

In [359]:
street_df_complete["closest_mrt"] = street_df_complete["coord_tuple"].apply(closest_mrt)

In [360]:
street_df_complete["closest_mrt"] = street_df_complete["closest_mrt"].astype(str)
street_df_complete["closest_mrt"] =street_df_complete["closest_mrt"].str.replace("[","")
street_df_complete["closest_mrt"] =street_df_complete["closest_mrt"].str.replace("]","")
street_df_complete["closest_mrt"] =street_df_complete["closest_mrt"].str.replace(" ","")
street_df_complete["closest_mrt"] =street_df_complete["closest_mrt"].str.replace(",","%2C")

# OneMap API

Documentation: https://www.onemap.gov.sg/apidocs/apidocs

This section will generate the route to take from a street's coordinates to its nearest train station.

In [361]:
key = "xxxxx"

In [364]:
def get_route_url(start, end):
    baseurl = "https://www.onemap.gov.sg/api/public/routingsvc/route?"
    routeType="pt"
    date="10-04-2023"
    time="07%3A35%3A00"
    mode="TRANSIT"
    numItineraries = "1"
    url = baseurl+"start="+start+"&end="+end+"&routeType="+routeType+"&date="+date+"&time="+time+"&mode="+mode+"&numItineraries="+numItineraries
    return url

# Travel time to nearest MRT

In [366]:
street_df_complete["route_url"] = street_df_complete.apply(lambda x: get_route_url(x["lat_long"], x["closest_mrt"]), axis=1)

#### API call to OneMap

In [377]:
oneMap_api = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 502, 503, 504, 400])
oneMap_api.mount('http://', HTTPAdapter(max_retries=retries))
headers = {"Authorization": key}

In [378]:
street_df_complete["route"] = street_df_complete["route_url"].apply(lambda x: oneMap_api.request("GET", x, headers=headers).json())

In [368]:
import pandas as pd
street_df_complete = pd.read_csv("complete_street_data.csv", header = 0)

In [369]:
import json
street_df_complete["route"] =  street_df_complete["route"].to_dict()
street_df_complete["route"].apply(lambda x: x.get("requestParameters", None) if isinstance(x, dict) else None)

In [464]:
street_df_complete["plan"] = street_df_complete["route"].apply(lambda x: x.get("plan", None) if isinstance(x, dict) else None)
street_df_complete["itinerary"] = street_df_complete["plan"].apply(lambda x: x.get("itineraries", None) if isinstance(x, dict) else None)
street_df_complete["travel_time"] = street_df_complete["itinerary"].apply(lambda x: x[0].get("duration", None) if isinstance(x, list) else None)
street_df_complete["walk_time"] = street_df_complete["itinerary"].apply(lambda x: x[0].get("walkTime", None) if isinstance(x, list) else None)


Export the CSV for use in main script

In [467]:
street_df_complete.to_csv("complete_street_data.csv")